# Setup CUDA

In [41]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [42]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-svzhkthr
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-svzhkthr
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done


In [43]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


#CUDA Parallelism Code

## Kode Percobaan Buat Test Setup Cuda (Bukan Buat Tucil CUDA)

In [44]:
%%cu
#include <iostream>
    int main() {
      std::cout << "Welcome To GeeksforGeeks\n";
      return 0;
    }

Welcome To GeeksforGeeks



In [45]:
%%cu
#include <cstdio>
#include <iostream>

using namespace std;

__global__ void maxi(int* a, int* b, int n)
{
	int block = 256 * blockIdx.x;
	int max = 0;

	for (int i = block; i < min(256 + block, n); i++) {

		if (max < a[i]) {
			max = a[i];
		}
	}
	b[blockIdx.x] = max;
}

int main()
{

	int n;
	n = 3 >> 2;
	int a[n];

	for (int i = 0; i < n; i++) {
		a[i] = rand() % n;
		cout << a[i] << "\t";
	}

	cudaEvent_t start, end;
	int *ad, *bd;
	int size = n * sizeof(int);
	cudaMalloc(&ad, size);
	cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
	int grids = ceil(n * 1.0f / 256.0f);
	cudaMalloc(&bd, grids * sizeof(int));

	dim3 grid(grids, 1);
	dim3 block(1, 1);

	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while (n > 1) {
		maxi<<<grids, block>>>(ad, bd, n);
		n = ceil(n * 1.0f / 256.0f);
		cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time = 0;
	cudaEventElapsedTime(&time, start, end);

	int ans[2];
	cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);

	cout << "The maximum element is : " << ans[0] << endl;

	cout << "The time required : ";
	cout << time << endl;
}


The maximum element is : 0
The time required : 0.003136



## Kode Cuda Buat Tucil

###Cara Run :
- upload "cuda.cu" yang telah kita buat
- upload test_case.txt (kalo udah butuh test case) => ini buat tucil
- jalankan command
- testing sample => !time ./cuda
- testing tucil => !time ./cuda < ./test_case/128.txt > output_cuda_128.txt

In [46]:
!nvcc cuda.cu -o cuda

gcc: error: cuda.cu: No such file or directory
gcc: warning: ‘-x c++’ after last input file has no effect
gcc: fatal error: no input files
compilation terminated.


In [47]:
# Command Buat Jalanin test sample cuda
!time ./cuda

# Command Buat Tucil Cuda (ini masih dugaan sih)
# !time ./cuda < ./test_case/128.txt > output_cuda_128.txt

/bin/bash: ./cuda: No such file or directory

real	0m0.001s
user	0m0.000s
sys	0m0.000s


In [54]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuComplex.h>

#define MAX_N 512
#ifndef M_PI
 # define M_PI		3.14159265358979323846
#endif

struct Matrix {
    int size;
    double mat[MAX_N][MAX_N];
};

struct FreqMatrix {
    int size;
    cuDoubleComplex mat[MAX_N][MAX_N];
};

__host__ __device__ static __inline__ cuDoubleComplex cuCexp(cuDoubleComplex x)
{
	double factor = exp(x.x);
	return make_cuDoubleComplex(factor * cos(x.y), factor * sin(x.y));
}

void readMatrix(struct Matrix *m) {
    scanf("%d", &(m->size));
    for (int i = 0; i < m->size; i++)
        for (int j = 0; j < m->size; j++)
            scanf("%lf", &(m->mat[i][j]));
}

cuDoubleComplex dft(struct Matrix *mat, int k, int l) {
    cuDoubleComplex element = make_cuDoubleComplex(0.0, 0.0);
    for (int m = 0; m < mat->size; m++) {
        for (int n = 0; n < mat->size; n++) {
            double arg = (k * m / (double)mat->size) + (l * n / (double)mat->size);
            cuDoubleComplex exponent = cuCexp(make_cuDoubleComplex(0.0, -2.0 * M_PI * arg));
            cuDoubleComplex mat_elem = make_cuDoubleComplex(mat->mat[m][n], 0.0);
            element = cuCadd(element, cuCmul(mat_elem, exponent));
        }
    }
    return cuCdiv(element, make_cuDoubleComplex(mat->size * mat->size, 0.0));
}

__global__ void sum_elements(struct FreqMatrix *mat, cuDoubleComplex *sum) {
    int col = blockDim.x * blockIdx.x + threadIdx.x;
    int row = blockDim.y * blockIdx.y + threadIdx.y;

    // posisi element matrix di bentuk row-major-order matrix
    int index = row * MAX_N + col;

    // jika row dan col masih berada pada jangkauan size matrix
    if (col < mat->size && row < mat->size) {
        cuDoubleComplex el = make_cuDoubleComplex(mat->mat[index], 0.0);
        *sum = cuCadd(*sum, el);
    }
}

int main(void) {
    struct Matrix source;
    struct FreqMatrix freq_domain;
    readMatrix(&source);
    freq_domain.size = source.size;
    cuDoubleComplex sum = make_cuDoubleComplex(0.0, 0.0);
    // copy sum
    cuDoubleComplex *d_sum;
    // copy matrix freq_domain
    struct FreqMatrix *d_freq_domain;

    // size dari copy sum dan copy matrix freq_domain
    size_t size_sum = MAX_N*sizeof(cuDoubleComplex);
    size_t size_mat = MAX_N*MAX_N*sizeof(struct FreqMatrix);

    // alokasi memori untuk device copy dari sum dan copy dari matrix freq_domain
    cudaMalloc((void **)&d_sum, size_sum);
    cudaMalloc((void **)&d_freq_domain, size_mat);

    // inialisasi freq_domain
    for (int k = 0; k < source.size; k++)
        for (int l = 0; l < source.size; l++)
            freq_domain.mat[k][l] = dft(&source, k, l);
    // copy sum dan matrix freq_domain ke devices/GPU
    cudaMemcpy(d_sum, &sum, size_sum, cudaMemcpyHostToDevice);
    cudaMemcpy(d_freq_domain, freq_domain, size_mat, cudaMemcpyHostToDevice);

    // init block and grid
    dim3 block(512, 512);    // dimensi block 512 * 512 * 1
    dim3 grid((int) ceil (MAX_N/block.x), (int)ceil(MAX_N/block.y));  // dimensi grid

    // GPU
    sum_elements<<<grid,block>>>(d_freq_domain, d_sum);

    // mengambil hasil sum dari GPU ke host
    cudaMemcpy(&sum, d_sum, size_sum, cudaMemcpyDeviceToHost);

    // sum /= source.size;
    sum = cuCdiv(sum, make_cuDoubleComplex(source.size, 0.0));
    printf("Average : (%lf, %lf)", cuCreal(sum), cuCimag(sum));

    // free alokasi
    cudaFree(d_sum);
    cudaFree(d_freq_domain);

    return 0;
}

/tmp/tmp3ugsrrh9/b8d09287-62e6-4a3d-b88d-6ab625000755.cu(56): error: argument of type "cuDoubleComplex *" is incompatible with parameter of type "double"

/tmp/tmp3ugsrrh9/b8d09287-62e6-4a3d-b88d-6ab625000755.cu(86): error: no suitable conversion function from "FreqMatrix" to "const void *" exists

2 errors detected in the compilation of "/tmp/tmp3ugsrrh9/b8d09287-62e6-4a3d-b88d-6ab625000755.cu".

